In [134]:
# -*- coding: utf-8 -*-
"""
Created on Tue May  7 14:29:49 2019

@author: Nathan Tefft

This script extracts data from the raw FARS data files to be used for replicating Levitt & Porter (2001). Selected variables are included, 
and the data definitions are harmonized across years. Accident, vehicle, and person dataframes are constructed and stored in csv files 
for later use in the replication.
"""

# This script has been validated for FARS datasets from 1982 to 2017

earliestYear=1982
latestYear = 2017

# install and import necessary packages
    # not all packages are standard installs
    # for example, us is not included in anaconda
    # you may need to install some of these packages in the command line

import os, numpy, pandas, shutil, us, zipfile

"""
   USER-DEFINED ATTRIBUTES 
      
1) The years over which FARS datasets are extracted and processed. 
    The default values are 1982 to 2017 
        **FARS begins in 1975, but the first year of imputed BAC is 1982
          If the user-provided first year is before 1982:
              -First year is reset to 1982
              -A warning message will appear to alert the user, but the script will not break.
         **FARS is updated annually, but this script is only validated through 2017
          If the user-provided last year is after 2017:
              -Last year is reset to 2017
              -A warning message will appear to alert the user, but the script will not break.     
          
    
2) The working directory.
    The user MUST set their own working directory before running the script. 
    We recommend the folder of the cloned GitHub repository.
        For example, set the working directory to "C:\\Users\\JoeEconomist\\GitHub\\lp"
    Data will then be placed into the subfolder .\data
"""

os.chdir("C:\\Users\\dunnr\\Documents\\GitHub\\lpdt")

# FARS data range
firstYear = 1982
lastYear = 1982


if firstYear>lastYear:
    print('User selected lastYear earlier than firstYear. firstYear has been set to ' + str(earliestYear) + ' and lastYear has been set to ' + str(latestYear) +'.')
    firstYear = earliestYear
    lastYear = latestYear
if firstYear < earliestYear:
    print('User selected firstYear prior to ' + str(earliestYear) + '. firstYear has been set to ' + str(earliestYear) +'.')
    firstYear = earliestYear
if lastYear > latestYear:
    print('User selected lastYear after ' + str(latestYear) + '. lastYear has been set to ' + str(latestYear) +'.')
    lastYear = latestYear

# load US state abbreviations for later merge
df_states = pandas.DataFrame.from_dict(us.states.mapping('fips', 'abbr'),orient='index',columns=['state_abbr'])
df_states = df_states[df_states.index.notnull()]
df_states.index = df_states.index.astype(int)

# Initialize analytic dataframe for the accident, vehicle, and person datasets

#fars_datasets = ['accident', 'vehicle', 'person', 'Miper']

fars_datasets = ['accident', 'vehicle', 'person', 'Miper']
dataset_ids = ['st_case','veh_no','per_no','per_no']

df_list={}
for dataset in fars_datasets:
    df_list[dataset]=pandas.DataFrame()

In [143]:
# loop over years to be included in the replication analysis
for yr in range(firstYear,lastYear+1): 
    print('Extracting data from ' + str(yr) + '.' )
    
    # extract accident, vehicle, person, and multiple imputation files
    zipfile.ZipFile('data\\FARS' + str(yr) + '.zip', 'r').extractall(path='data\\extracted')
    # UTF-8 encoding errors are ignored because they don't impact the relevant variables
        
    df_list_yr={}
    index_list=['year']
    for (dataset, id) in zip(fars_datasets,dataset_ids):
        file = open('data\\extracted\\' + dataset + '.csv', errors='ignore')
        df_list_yr[dataset]=pandas.read_csv(file)
        file.close()
    
        print(dataset + str(df_list_yr[dataset].shape))
        df_list_yr[dataset].columns = df_list_yr[dataset].columns.str.lower() # make all columns lowercase
        df_list_yr[dataset]['year']=numpy.full(len(df_list_yr[dataset].index), yr) # standardize the year variable to 4 digits
        
        if not dataset == 'Miper':
            index_list.append(id)
        print(index_list)
        df_list_yr[dataset][index_list] = df_list_yr[dataset][index_list].astype('int') # set the indices as integers
        df_list_yr[dataset].set_index(index_list, inplace=True) # set the multiindex
        df_list_yr[dataset].index.set_names(index_list, inplace=True)  
    '''    
    # BAC MI dataset
    dataset="Miper"
    file = open('data\\extracted\\' + dataset + '.csv', errors='ignore')
    df_list_yr[dataset]=pandas.read_csv(file)
    file.close()
    
    print(dataset + str(df_list_yr[dataset].shape))
    df_list_yr[dataset].columns = df_list_yr[dataset].columns.str.lower() # make all columns lowercase
    df_list_yr[dataset]['year']=numpy.full(len(df_list_yr[dataset].index), yr) # standardize the year variable to 4 digits
    df_list_yr[dataset][index_list] = df_list_yr[dataset][index_list].astype('int') # set the indices as integers
    df_list_yr[dataset].set_index(index_list, inplace=True) # set the multiindex
    df_list_yr[dataset].index.set_names(index_list, inplace=True)
    '''    
    shutil.rmtree(path='data\\extracted') # clean up temporary extractions folder

Extracting data from 1982.
accident(39092, 47)
['year', 'st_case']
vehicle(56455, 73)
['year', 'st_case', 'veh_no']
person(102120, 59)
['year', 'st_case', 'veh_no', 'per_no']
Miper(65070, 14)
['year', 'st_case', 'veh_no', 'per_no']


In [124]:
    # manipulating vehicle data
    if yr <= 2008: 
        df_list_yr['vehicle']['occupants'] = df_list_yr['vehicle']['ocupants']
    else:
        df_list_yr['vehicle']['occupants'] = df_list_yr['vehicle']['numoccs']
    if yr <= 2015:
        df_list_yr['vehicle'].loc[df_list_yr['vehicle'].occupants>=99, 'occupants'] = numpy.nan
    else:
        df_list_yr['vehicle'].loc[df_list_yr['vehicle'].occupants>=97, 'occupants'] = numpy.nan	
    for vt in ['acc','sus','dwi','spd','oth']:
        df_list_yr['vehicle'].loc[df_list_yr['vehicle']['prev_' + vt] > 97, 'prev_' + vt] = numpy.nan # previous violations

    # keep relevant vehicle variables and append to vehicle dataframe
    df_list_yr['vehicle'] = df_list_yr['vehicle'][['prev_acc','prev_sus','prev_dwi','prev_spd','prev_oth','dr_drink','occupants']]
    print('Count of vehicles: ' + str(len(df_list_yr['vehicle'])))
    df_list['vehicle'] = df_list['vehicle'].append(df_list_yr['vehicle'])

Count of vehicles: 56455


In [125]:
    # manipulate person variables
    
    #standardize alcohol test result
    if yr <= 1990: 
        df_list_yr['person']['alcohol_test_result'] = df_list_yr['person']['test_res']
    else:
        df_list_yr['person']['alcohol_test_result'] = df_list_yr['person']['alc_res']
    
    if yr >= 2015:
        df_list_yr['person']['alcohol_test_result'] = df_list_yr['person']['alcohol_test_result']/10
    
    df_list_yr['person'].loc[df_list_yr['person'].alcohol_test_result>=95, 'alcohol_test_result'] = numpy.nan    
    
    
    for vn in ['alc_det','atst_typ','race']: # create variables if necessary
        if vn not in df_list_yr['person'].columns:
            df_list_yr['person'][vn] = numpy.nan
    
    if yr <= 2008:
        df_list_yr['person'].loc[df_list_yr['person'].age==99, 'age'] = numpy.nan # age
    else:
        df_list_yr['person'].loc[df_list_yr['person'].age>=998, 'age'] = numpy.nan # age
    
    df_list_yr['person']['age_lt15'] = df_list_yr['person']['age'] < 15 # less than 15 defined as child for our purposes
    df_list_yr['person'].loc[df_list_yr['person'].sex.isin([8,9]), 'sex'] = numpy.nan # sex
    df_list_yr['person'].loc[df_list_yr['person'].race==99, 'race'] = numpy.nan # race
    df_list_yr['person'].loc[df_list_yr['person'].seat_pos>=98, 'seat_pos'] = numpy.nan # seat position

In [126]:
    # clean mulptiple imputation variables, e.g. harmonize names, ensure correct datatypes, and record missing variables 
    df_list_yr['Miper'] = df_list_yr['Miper'].rename(columns={'p1':'mibac1','p2':'mibac2','p3':'mibac3','p4':'mibac4','p5':'mibac5','p6':'mibac6','p7':'mibac7','p8':'mibac8','p9':'mibac9','p10':'mibac10'}) # rename bac columns    
    df_list_yr['person'] = df_list_yr['person'].merge(df_list_yr['Miper'],how='left',on=['year','st_case','veh_no','per_no']) # merge multiply imputed bac values into person dataframe
    
    # keep relevant person variables and append to person dataframe
    df_list_yr['person'] = df_list_yr['person'][['seat_pos','drinking','alc_det','atst_typ','alcohol_test_result','race','age','age_lt15','sex','mibac1','mibac2','mibac3','mibac4','mibac5','mibac6','mibac7','mibac8','mibac9','mibac10']]
    print('Count of persons: ' + str(len(df_list_yr['person'])))
    df_list['person'] = df_list['person'].append(df_list_yr['person'])

Count of persons: 102120


In [129]:
# summarize the constructed dataframes and save to csv files
if not os.path.exists('replication\\data'):
    os.makedirs('replication\\data')
for dfn in fars_datasets:
    print('Describing dataframe ' + dfn)
    print(df_list[dfn].describe())
    df_list[dfn].to_csv('replication\\data\\df_' + dfn + '.csv')
    

Describing dataframe accident
              state       quarter      day_week          hour       persons
count  39092.000000  39092.000000  39090.000000  38879.000000  39092.000000
mean      28.140157      2.593344      4.240982     12.444224      2.612299
std       15.973887      1.077992      2.151909      7.684351      1.748127
min        1.000000      1.000000      1.000000      0.000000      1.000000
25%       13.000000      2.000000      2.000000      5.000000      2.000000
50%       28.000000      3.000000      4.000000     14.000000      2.000000
75%       42.000000      4.000000      6.000000     19.000000      3.000000
max       56.000000      4.000000      7.000000     23.000000     66.000000
Describing dataframe vehicle
           prev_acc      prev_sus      prev_dwi      prev_spd      prev_oth  \
count  53133.000000  53132.000000  53132.000000  53133.000000  53133.000000   
mean       0.254663      0.132952      0.052962      0.533661      0.355147   
std        0.586415 

In [52]:
# loop over years to be included in the replication analysis
for yr in range(firstYear,lastYear+1): 
    print('Extracting data from ' + str(yr) + '.' )
    
    # extract accident, vehicle, person, and multiple imputation files
    zipfile.ZipFile('data\\FARS' + str(yr) + '.zip', 'r').extractall(path='data\\extracted')
    # UTF-8 encoding errors are ignored because they don't impact the relevant variables
    #, 'vehicle', 'person', 'Miper'
    
    df_list_yr={}
    for (dataset, id) in zip(fars_datasets,dataset_id):
        file = open('data\\extracted\\' + dataset + '.csv', errors='ignore')
        df_list_yr[dataset]=pandas.read_csv(file)
        file.close()
        
        print(dataset + str(df_list_yr[dataset].shape))
        df_list_yr[dataset].columns = df_list_yr[dataset].columns.str.lower() 
    
    shutil.rmtree(path='data\\extracted') # clean up temporary extractions folde
            
    
    df_list_yr['accident']['st_case'] = df_list_yr['accident']['st_case'].astype('int')
    df_list_yr['accident'].set_index([numpy.full(len(df_list_yr['accident'].index), yr),'st_case'],inplace=True) # set the multiindex as year and st_case
    df_list_yr['accident'].index.set_names(['year','st_case'], inplace=True) # set the multiindex names
    df_list_yr['accident'].loc[df_list_yr['accident'].hour==99, 'hour'] = numpy.nan
    df_list_yr['accident'].loc[df_list_yr['accident'].hour==24, 'hour'] = 0
    df_list_yr['accident'].loc[df_list_yr['accident'].day_week==9, 'day_week'] = numpy.nan
    df_list_yr['accident']['quarter'] = numpy.ceil(df_list_yr['accident']['month']/3) # create quarter variable
    df_list_yr['accident'] = df_list_yr['accident'].merge(df_states,how='left',left_on='state',right_index=True) # merge in state abbreviations

    # keep relevant accident variables and append to accident dataframe
    df_list_yr['accident'] = df_list_yr['accident'][['state','state_abbr','quarter','day_week','hour','persons']].copy()
    print('Count of crashes: ' + str(len(df_list_yr['accident'])))
    df_list['accident'] = df_list['accident'].append(df_list_yr['accident'])
    print('Count of crashes: ' + str(len(df_list['accident'])))
      
    

Extracting data from 1982.


NameError: name 'dataset_id' is not defined

In [66]:
# loop over years to be included in the replication analysis
for yr in range(firstYear,lastYear+1): 
    print('Extracting data from ' + str(yr) + '.' )
    
    # extract accident, vehicle, person, and multiple imputation files
    zipfile.ZipFile('data\\FARS' + str(yr) + '.zip', 'r').extractall(path='data\\extracted')
    # UTF-8 encoding errors are ignored because they don't impact the relevant variables
    #, 'vehicle', 'person', 'Miper'
    
    df_list_yr={}
    index_list=[]
    for (dataset, id) in zip(fars_datasets,dataset_ids):
        file = open('data\\extracted\\' + dataset + '.csv', errors='ignore')
        df_list_yr[dataset]=pandas.read_csv(file)
        file.close()
        #print(dataset + str(df_list_yr[dataset].shape))
        #df_list_yr[dataset].columns = df_list_yr[dataset].columns.str.lower() 
    
        print(dataset + str(df_list_yr[dataset].shape))
        df_list_yr[dataset].columns = df_list_yr[dataset].columns.str.lower()
        df_list_yr[dataset]['year']=numpy.full(len(df_list_yr[dataset].index), yr)
        index_list.append(id)
        print(index_list)
        df_list_yr[dataset][index_list] = df_list_yr[dataset][index_list].astype('int')
        #df_list_year[dataset].set_index(yr,index_list)
        #df_list_yr[dataset].set_index([numpy.full(len(df_list_yr[dataset].index), yr), index_list],inplace=True) # set the multiindex as year and st_case
        #df_list_yr[dataset].index.set_names(['year',index_list], inplace=True) # set the multiindex names
    
    shutil.rmtree(path='data\\extracted') # clean up temporary extractions folder
    
    # Accident dataset manipulation
    
    df_accident_yr = df_list_yr['accident'][['year','st_case','state', 'day_week','hour','month','persons']].copy()
    df_accident_yr.loc[df_accident_yr.hour==99, 'hour'] = numpy.nan
    df_accident_yr.loc[df_accident_yr.hour==24, 'hour'] = 0
    df_accident_yr.loc[df_accident_yr.day_week==9, 'day_week'] = numpy.nan
    df_accident_yr['quarter'] = numpy.ceil(df_accident_yr['month']/3) # create quarter variable
    df_accident_yr = df_accident_yr.merge(df_states,how='left',left_on='state',right_index=True) # merge in state abbreviations

    print('Count of crashes: ' + str(len(df_accident_yr)))
    
    df_list['accident'] = df_list['accident'].append(df_accident_yr)

Extracting data from 1982.
accident(39092, 47)
['st_case']


ValueError: all arrays must be same length